In [62]:
import os
import os.path as op
import glob
import numpy as np
import multiprocessing
from nipype.interfaces import fsl

In [29]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['mag-SC']
runs = map(str,range(1,5))

In [52]:
#get cortical areas from Yeo atlas
path = home_dir + '/data/fsaverage/label/'
# cortical_areas =  glob.glob(path + '*.7Net*')
# cortical_areas = [x.split('/')[-1] for x in cortical_areas]

#get cortical areas from Michael's parcellation
# cortical_areas =  glob.glob(path + '*yeo*')
# cortical_areas = [x.split('/')[-1] for x in cortical_areas]
# print cortical_areas

#get vmpfc roi
cortical_areas =  glob.glob(path + '*vmpfc*')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]
cortical_areas = [cortical_areas[0]]
cortical_areas

['rh.vmpfc_bartre.label']

In [53]:
def make_mask(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    hemi = roi.split('.')[0]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           hemi + '.' + roi_id,
            '-label',
           roi_id,
           '-sample cortex',
#            '-proj frac 0 1 .1',
           '-hemi',
           hemi,
           '-debug',
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [56]:
def make_mask_no_hemi(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    hemi = roi.split('.')[0]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           roi_id,
            '-label',
           roi_id,
           '-sample cortex',
           '-debug',
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [57]:
in_tuples = []
for sub in subs:
    for roi in cortical_areas:
        in_tuples.append((sub,roi))

In [58]:
pool = multiprocessing.Pool(50)
pool.map(make_mask_no_hemi,in_tuples)
pool.terminate()
pool.join()

# Map MNI ROI back into EPI space

In [80]:
rois = ['figure3D_vmpfc_bartre']
exp = 'mag'
#warp group statistical image back into subject space
for sub in subs: 
    if sub != 'subject03':
        #set paths
        mask_path = home_dir + '/data/' + sub
        out_dir = mask_path + '/masks/'

        #copy warpfile to make ANTs happy
        warp = mask_path + '/normalization/inverse_warpfield.nii.gz'
        warpfield = mask_path + '/normalization/InverseWarp.nii.gz'
        cmd = ['ln','-s',warp, warpfield]
        os.system(' '.join(cmd))

        for roi in rois:
            im =  home_dir + '/rois/' + roi + '.nii.gz'

            #warp to subject space
            out_warp = out_dir + roi + '_anat_space.nii.gz'
            ref_file = fsl.Info.standard_image("avg152T1_brain.nii.gz")
            affine = home_dir + '/data/' + sub + '/normalization/affine.txt'
            anat = home_dir + '/data/' + sub + '/anat/brain_out.nii.gz'
            if os.path.exists(out_warp):
                os.remove(out_warp)
            cmd_str = ['WarpImageMultiTransform','3',im,out_warp,'-i',affine,warpfield,
                       '-R',anat]
            cmd_str = ' '.join(cmd_str)
            os.system(cmd_str)

            #register to functional space
            sub_path = home_dir + 'analysis/' + exp + '/' + sub

            #register to func
            transform = sub_path + '/preproc/run_1/func2anat_tkreg.dat'
            mean_func = sub_path + '/preproc/run_1/mean_func.nii.gz'
            out_rigid = out_dir + exp + '_' + roi + '_func_space.nii.gz'
            if os.path.exists(out_rigid):
                os.remove(out_rigid)
            cmd_line = ['mri_vol2vol','--mov',mean_func,'--targ',out_warp,'--reg',transform,'--o',out_rigid,
                        '--no-save-reg','--inv']
            os.system(' '.join(cmd_line))

            #binarize
            out_mask = out_dir + exp + '_' + roi + '_mask.nii.gz'
            if os.path.exists(out_mask):
                os.remove(out_mask)
            cmd = ['fslmaths',out_rigid,'-thr','.1','-bin',out_mask]
            os.system(' '.join(cmd))


In [76]:
for sub in subs:
    anat = '/home/iballard/mag/analysis/workingdir/antswarp/_subject_id_' + sub + '/convertbrain/brain_out.nii.gz'
    new = '/home/iballard/mag/data/' + sub + '/anat/brain_out.nii.gz'
    
    os.rename(anat,new)

In [82]:
for sub in subs:
    roi_path = '/home/iballard/mag/data/' + sub + '/masks/'
    
    cmd = ['fslmaths',roi_path + 'lh.vmpfc_bartre','-add',
           roi_path + 'rh.vmpfc_bartre', roi_path + 'temp']
    os.system(' '.join(cmd))
    
    cmd = ['fslmaths',roi_path + 'temp','-bin',
           roi_path + 'vmpfc_freesurfer']
    os.system(' '.join(cmd))
    